In [1]:
import pandas as pd
import polars as pl
import numpy as np
import os
import sys
from datetime import datetime
import winsound
import ctypes

# Adicionando caminhos dos módulos (ajuste se necessário)
sys.path.insert(0, r'C:\Scripts\modules_azure\database')
sys.path.insert(0, r'C:\Scripts\modules_azure\parameters')

# Importando os módulos novos
from connection_azure import Connect
from azure_loader import AzureLoader
from parametros import Parametros


SCHEMA_DEFAULT = "dbo" 
CAMINHO_RELATORIOS = Parametros.caminho_completo_atual()

<h3>Definição dos parametros</h3>

In [2]:
if not CAMINHO_RELATORIOS:
    print("Erro: Pasta de relatórios de hoje não encontrada.")
    sys.exit()

print(f"Diretório base: {CAMINHO_RELATORIOS}")

Diretório base: C:\Scripts\relatórios\\06-02-2026\0848\


<h1><center>Migração das tabelas</center></h1>

#####  Tabela times

In [3]:
print("Subindo a tabela de Times")
try:
    times = pl.read_excel("C:\\Scripts\\times_atria.xlsx")
    times = times.with_columns(pl.col("Assessor").str.to_uppercase())
    
    # Envio com AzureLoader
    AzureLoader.enviar_df(
        df=times.to_pandas(),
        nome_tabela="times_nova_empresa",
        col_pk="Assessor",
        if_exists="replace",
        schema=SCHEMA_DEFAULT
    )
except Exception as e:
    print(f"Erro em Times: {e}")

Subindo a tabela de Times
[AzureLoader] Subindo tabela: dbo.times_nova_empresa (15 linhas)...
[AzureLoader] Chunksize calculado: 696 linhas/lote (Colunas: 3).
[AzureLoader] Configurando Primary Key: Assessor
[AzureLoader] Concluido: times_nova_empresa atualizada.


##### Tipo clientes

In [4]:
print("Classificando Tipo de Clientes")
try:
    # Lê as tabelas atuais do Azure para comparação
    base_btg_db = AzureLoader.ler_tabela("base_btg", schema=SCHEMA_DEFAULT)
    tipo_clientes_hist = AzureLoader.ler_tabela("tipo_clientes", schema=SCHEMA_DEFAULT)
    
    if not base_btg_db.empty:
        tipo_clientes = base_btg_db.loc[:, ["Conta", "Tipo"]]
        
        # Filtra apenas contas que NÃO estão no histórico
        if not tipo_clientes_hist.empty:
            tipo_clientes = tipo_clientes[~tipo_clientes['Conta'].isin(tipo_clientes_hist['Conta'])]
        
        if not tipo_clientes.empty:
            tipo_clientes.loc[tipo_clientes['Tipo'].isnull(), "Tipo"] = 'Offshore'
            
            # Append apenas dos novos
            AzureLoader.enviar_df(
                df=tipo_clientes,
                nome_tabela="tipo_clientes",
                if_exists="append", # Mantém o histórico, adiciona novos
                schema=SCHEMA_DEFAULT
                # Nota: Não passamos col_pk aqui pois é append, a tabela já existe
            )
        else:
            print("Nenhum cliente novo para classificar.")
    else:
        print("Tabela base_btg vazia ou inexistente no banco.")

except Exception as e:
    print(f"Erro em Tipo Clientes: {e}")

Classificando Tipo de Clientes
[AzureLoader] Lendo: dbo.base_btg...
[AzureLoader] Lendo: dbo.tipo_clientes...
Nenhum cliente novo para classificar.


##### Base BTG

In [5]:
print("Atualizando Base BTG e Offshore")
try:
    # Leitura do Excel do dia
    base = pd.read_excel(os.path.join(CAMINHO_RELATORIOS, "base_btg.xlsx"), header=2)
    base.rename(columns={"E-Mail Comunicação": "E-mail"}, inplace=True)
    
    # Limpeza de nomes de colunas (R$)
    base.columns = [col.replace(" (R$)", "") for col in base.columns]
    base['Conta'] = base['Conta'].astype(str).str.strip()
    
    # Tratamentos de texto
    base['Assessor'] = base['Assessor'].astype(str).str.upper()
    
    # Normalização Faixa Cliente
    base.loc[base['Faixa Cliente'].isin(["Ate 50K", "Entre 50k e 100k", "Entre 100k e 300k"]), "Faixa Cliente"] = "Até 300k"
    
    # Processamento Offshore
    offshore = pd.read_excel(r"C:\Scripts\historico_auc\AuC Offshore.xlsx", sheet_name='AuC Offshore')
    offshore = offshore.loc[:, ["Nome", "Conta", "AUC BRL", "Assessor"]]
    offshore.rename(columns={"AUC BRL": "PL Total"}, inplace=True)
    offshore['Conta'] = offshore['Conta'].astype(str).str.strip()
    offshore['Assessor'] = offshore['Assessor'].astype(str).str.upper()
    
    # Upload PL Offshore
    AzureLoader.enviar_df(
        df=offshore,
        nome_tabela="pl_offshore",
        col_pk="Conta",
        if_exists="replace",
        schema=SCHEMA_DEFAULT
    )
    
    # Concatenação e Ajustes Finais Base
    base_full = pd.concat([offshore, base], axis=0)
    
    # Ajuste Assessores Reais
    base_full.loc[base_full['Assessor'] == "MURILO LUIZ SILVA GINO", "Assessor"] = "IZADORA VILLELA FREITAS"
    base_full.loc[base_full['Assessor'].str.contains("GABRIEL GUERRERO TORRES FONSECA", na=False), "Assessor"] = "MARCOS SOARES PEREIRA FILHO"
    
    base_full.drop_duplicates(subset="Conta", keep='first', inplace=True)
    base_full.rename(columns={"Perfil de Cliente": "Perfil do Cliente"}, inplace=True)
    
    # Ajuste Rodrigo
    base_full.loc[base_full['Assessor'].isin(["Rodrigo de Mello D?Elia", "Rodrigo de Mello DElia", "RODRIGO DE MELLO DELIA"]), "Assessor"] = "RODRIGO DE MELLO D’ELIA"
    
    # Upload Base BTG Completa
    AzureLoader.enviar_df(
        df=base_full,
        nome_tabela="base_btg",
        col_pk="Conta",
        if_exists="replace",
        schema=SCHEMA_DEFAULT
    )

except Exception as e:
    print(f"Erro em Base BTG/Offshore: {e}")

Atualizando Base BTG e Offshore
[AzureLoader] Subindo tabela: dbo.pl_offshore (57 linhas)...
[AzureLoader] Chunksize calculado: 522 linhas/lote (Colunas: 4).
[AzureLoader] Configurando Primary Key: Conta
[AzureLoader] Concluido: pl_offshore atualizada.
[AzureLoader] Subindo tabela: dbo.base_btg (1244 linhas)...
[AzureLoader] Chunksize calculado: 46 linhas/lote (Colunas: 45).
[AzureLoader] Configurando Primary Key: Conta
[AzureLoader] Concluido: base_btg atualizada.


#### Acompanhamento de custódia


In [6]:
try:
    arquivos = os.listdir(CAMINHO_RELATORIOS)
    arquivo_custodia = [f for f in arquivos if 'acompanhamento-de-custodia' in f][0]
    
    custodia = pd.read_excel(os.path.join(CAMINHO_RELATORIOS, arquivo_custodia))
    
    AzureLoader.enviar_df(
        df=custodia,
        nome_tabela="acompanhamento_custodia",
        if_exists="replace",
        schema=SCHEMA_DEFAULT
    )
except Exception as e:
    print(f"Erro em Custódia: {e}")

Erro em Custódia: list index out of range


##### Nomes completos

In [7]:
try:
    caminho_nomes = r"C:\Scripts\nomes_clientes\Nomes_Clientes2.xlsx"
    nomes_clientes = pd.read_excel(caminho_nomes)
    nomes_clientes.rename(columns={"Código": "Conta"}, inplace=True)
    nomes_clientes['Conta'] = nomes_clientes['Conta'].astype(str).str.strip()
    nomes_clientes = nomes_clientes[~nomes_clientes['Nome'].isna()]
    
    # Merge com a base recém tratada (usando base_full da memória)
    temp_base = base_full[['Conta', 'Nome']].copy()
    nomes_clientes = pd.merge(temp_base, nomes_clientes, on="Conta", how='outer', suffixes=('_novo', '_antigo'))
    
    # Preenchimento de nomes
    nomes_clientes['Nome'] = nomes_clientes['Nome_antigo'].fillna(nomes_clientes['Nome_novo'])
    nomes_clientes = nomes_clientes[['Conta', 'Nome']] # Mantém apenas colunas finais
    
    nomes_clientes['Conta'] = nomes_clientes['Conta'].astype(str)
    nomes_clientes.drop_duplicates(subset='Conta', inplace=True)
    
    # Salva Excel e Banco
    nomes_clientes.to_excel(caminho_nomes, header=True, index=False)
    
    AzureLoader.enviar_df(
        df=nomes_clientes,
        nome_tabela="nomes_clientes",
        col_pk="Conta",
        if_exists="replace",
        schema=SCHEMA_DEFAULT
    )
except Exception as e:
    print(f"Erro em Nomes Clientes: {e}")

[AzureLoader] Subindo tabela: dbo.nomes_clientes (2980 linhas)...
[AzureLoader] Chunksize calculado: 1000 linhas/lote (Colunas: 2).
[AzureLoader] Configurando Primary Key: Conta
[AzureLoader] Concluido: nomes_clientes atualizada.


<h3>Saldo CC</h3>

In [8]:
try:
    saldo_cc = pd.read_excel(os.path.join(CAMINHO_RELATORIOS, "saldo_cc.xlsx"), header=2)
    saldo_cc.columns = [col.replace(" (R$)", "") for col in saldo_cc.columns]
    saldo_cc.rename(columns={"Saldo": "SALDO"}, inplace=True)
    
    saldo_cc = saldo_cc.loc[:, ["Conta", "SALDO"]]
    saldo_cc['Conta'] = saldo_cc['Conta'].astype(str).str.strip()
    
    # Merge com base para pegar Assessor
    saldo_cc = pd.merge(saldo_cc, base_full[['Assessor', 'Conta']], on='Conta', how='left')
    
    AzureLoader.enviar_df(
        df=saldo_cc,
        nome_tabela="saldo_conta_corrente",
        col_pk="Conta",
        if_exists="replace",
        schema=SCHEMA_DEFAULT
    )
except Exception as e:
    print(f"Erro em Saldo CC: {e}")

[AzureLoader] Subindo tabela: dbo.saldo_conta_corrente (625 linhas)...
[AzureLoader] Chunksize calculado: 696 linhas/lote (Colunas: 3).
[AzureLoader] Configurando Primary Key: Conta
[AzureLoader] Concluido: saldo_conta_corrente atualizada.


<h2>Posição</h2>

In [9]:
try:
    posicao = pd.read_excel(os.path.join(CAMINHO_RELATORIOS, "posição.xlsx"), header=2)
    posicao.columns = [col.replace(" (R$)", "") for col in posicao.columns]
    
    posicao.rename(columns={'CONTA': 'Conta', 'Submercado': 'Sub Mercado', "Vencimento": "VENCIMENTO", "Escritório": "ESCRITÓRIO"}, inplace=True)
    posicao['Conta'] = posicao['Conta'].astype(str).str.strip()
    posicao['VENCIMENTO'] = pd.to_datetime(posicao['VENCIMENTO'], format="%Y-%m-%d", errors='coerce')
    
    posicao = pd.merge(posicao, base_full[['Conta', 'Assessor']], on='Conta', how='left')
    posicao.rename(columns={"IR": "Soma de IR", "IOF": "Soma de IOF"}, inplace=True)
    if "ESCRITÓRIO" in posicao.columns:
        posicao.drop("ESCRITÓRIO", axis=1, inplace=True)
    
    # Lógica de Setores
    setores = pd.read_excel(r"C:\Scripts\setores_ativos\setores.xlsx")
    posicao['Setor'] = ''
    posicao['Subsetor'] = ''
    
    # Otimização da iteração (usando map em vez de loop lento)
    map_setor = dict(zip(setores['Ativo'], setores['SETOR_ECONÔMICO']))
    map_subsetor = dict(zip(setores['Ativo'], setores['SUBSETOR']))
    # Nota: A lógica original varre emissor também, mantendo loop simplificado ou usando apply
    for index, row in setores.iterrows():
        mask = (posicao['Ativo'] == row['Ativo']) | (posicao['Emissor'] == row['Emissor'])
        posicao.loc[mask, 'Setor'] = row['SETOR_ECONÔMICO']
        posicao.loc[mask, 'Subsetor'] = row['SUBSETOR']

    AzureLoader.enviar_df(
        df=posicao,
        nome_tabela="posicao",
        if_exists="replace",
        schema=SCHEMA_DEFAULT
    )
except Exception as e:
    print(f"Erro em Posição: {e}")

[AzureLoader] Subindo tabela: dbo.posicao (16643 linhas)...
[AzureLoader] Chunksize calculado: 83 linhas/lote (Colunas: 25).
[AzureLoader] Concluido: posicao atualizada.


<h2>Renda fixa</h2>

In [10]:
try:
    rv = pd.read_excel(os.path.join(CAMINHO_RELATORIOS, "renda_variavel.xlsx"), header=2)
    rv.columns = [col.replace(" (R$)", "") for col in rv.columns]
    rv.rename(columns={"Data Vencimento": "DATA VENCIMENTO", "'DL_D_ContaAssessor'[NR_CONTA]": "Conta"}, inplace=True)
    
    rv['DATA VENCIMENTO'] = pd.to_datetime(rv['DATA VENCIMENTO'], format=("%Y-%m-%d"), errors='coerce')
    rv['Conta'] = rv['Conta'].astype(str).str.strip()
    
    # Cálculos
    rv['Valor Bruto Inicial'] = rv['Preço Médio'] * rv['Quantidade']
    rv['Performance'] = ((rv['Valor Bruto'] - rv['Valor Bruto Inicial']) / rv['Valor Bruto Inicial']) * 100
    rv.loc[rv['Performance'] == np.inf, "Performance"] = 0
    
    AzureLoader.enviar_df(df=rv, nome_tabela="renda_variavel", if_exists="replace", schema=SCHEMA_DEFAULT)
except Exception as e:
    print(f"Erro em Renda Variável: {e}")

[AzureLoader] Subindo tabela: dbo.renda_variavel (1776 linhas)...
[AzureLoader] Chunksize calculado: 116 linhas/lote (Colunas: 18).
[AzureLoader] Concluido: renda_variavel atualizada.


<h2>Renda varíavel</h2>

In [11]:
try:
    rv = pd.read_excel(os.path.join(CAMINHO_RELATORIOS, "renda_variavel.xlsx"), header=2)
    rv.columns = [col.replace(" (R$)", "") for col in rv.columns]
    rv.rename(columns={"Data Vencimento": "DATA VENCIMENTO", "'DL_D_ContaAssessor'[NR_CONTA]": "Conta"}, inplace=True)
    
    rv['DATA VENCIMENTO'] = pd.to_datetime(rv['DATA VENCIMENTO'], format=("%Y-%m-%d"), errors='coerce')
    rv['Conta'] = rv['Conta'].astype(str).str.strip()
    
    # Cálculos
    rv['Valor Bruto Inicial'] = rv['Preço Médio'] * rv['Quantidade']
    rv['Performance'] = ((rv['Valor Bruto'] - rv['Valor Bruto Inicial']) / rv['Valor Bruto Inicial']) * 100
    rv.loc[rv['Performance'] == np.inf, "Performance"] = 0
    
    AzureLoader.enviar_df(df=rv, nome_tabela="renda_variavel", if_exists="replace", schema=SCHEMA_DEFAULT)
except Exception as e:
    print(f"Erro em Renda Variável: {e}")

[AzureLoader] Subindo tabela: dbo.renda_variavel (1776 linhas)...
[AzureLoader] Chunksize calculado: 116 linhas/lote (Colunas: 18).
[AzureLoader] Concluido: renda_variavel atualizada.


<h2>Rentabilidade</h2>

In [12]:
try:
    rent = pd.read_excel(os.path.join(CAMINHO_RELATORIOS, "rentabilidade.xlsx"), header=2)
    rent['Conta'] = rent['Conta'].astype(str).str.strip()
    cols_rent = ["Conta", "Data", 'Profitability mtd', 'Profitability last 12 months accumulated',
                 'Profitability last 6 months accumulated', 'Profitability last 3 months accumulated', 
                 'Profitability ytd', 'Modelo de Rentabilidade']
    rent = rent.loc[:, cols_rent]
    
    AzureLoader.enviar_df(df=rent, nome_tabela="rentabilidade", col_pk="Conta", if_exists="replace", schema=SCHEMA_DEFAULT)
except Exception as e:
    print(f"Erro em Rentabilidade: {e}")

[AzureLoader] Subindo tabela: dbo.rentabilidade (1085 linhas)...
[AzureLoader] Chunksize calculado: 261 linhas/lote (Colunas: 8).
[AzureLoader] Configurando Primary Key: Conta
[AzureLoader] Concluido: rentabilidade atualizada.


#### PL Base

In [13]:
try:
    # 1. Carrega Histórico do Banco
    pl_base_hist = AzureLoader.ler_tabela("PL Base", schema=SCHEMA_DEFAULT)
    
    # 2. Prepara Base Atual (usa base_full já carregada na memória no passo 3)
    base_hoje = base_full.copy() # Usa a base com offshore já concatenada
    base_hoje = base_hoje[['Assessor', 'Conta', 'PL Total']]
    base_hoje.rename(columns={'PL Total': 'PL', 'Conta': 'CONTA'}, inplace=True)
    base_hoje['Mês'] = datetime.today().strftime("%Y-%m-%d")
    
    # 3. Prepara Offshore Histórico (tabela auxiliar de histórico offshore)
    pl_offshore_hist = AzureLoader.ler_tabela("offshore_adicionar_pl_mes_vigente", schema=SCHEMA_DEFAULT)
    
    # 4. Atualiza tabela auxiliar Offshore
    offshore_current = offshore.copy() # Do passo 3
    offshore_current.rename(columns={"PL Total": "PL", "Conta": "CONTA"}, inplace=True)
    offshore_current['Mês'] = datetime.today().strftime("%Y-%m-%d")
    if "Nome" in offshore_current.columns:
        offshore_current.drop("Nome", axis=1, inplace=True)
        
    # Remove mês atual do histórico para não duplicar e adiciona o novo
    data_corte_mes = datetime.today().strftime("%Y-%m-01")
    
    if not pl_offshore_hist.empty:
        # Garante formato string YYYY-MM-DD para comparação
        # Ajuste dependendo de como o pandas leu do SQL (datetime ou str)
        if pd.api.types.is_datetime64_any_dtype(pl_offshore_hist['Mês']):
             pl_offshore_hist = pl_offshore_hist[pl_offshore_hist['Mês'] < datetime.today().replace(day=1)]
             pl_offshore_hist['Mês'] = pl_offshore_hist['Mês'].dt.strftime("%Y-%m-%d")
        else:
             pl_offshore_hist = pl_offshore_hist[pl_offshore_hist['Mês'] < data_corte_mes]

    offshore_atualizado = pd.concat([offshore_current, pl_offshore_hist], axis=0)
    
    AzureLoader.enviar_df(
        df=offshore_atualizado, 
        nome_tabela="offshore_adicionar_pl_mes_vigente", 
        if_exists="replace", 
        schema=SCHEMA_DEFAULT
    )
    
    # 5. Consolidação Final do PL Base
    # Filtra histórico removendo mês atual
    if not pl_base_hist.empty:
         if pd.api.types.is_datetime64_any_dtype(pl_base_hist['Mês']):
             pl_base_hist = pl_base_hist[pl_base_hist['Mês'] < datetime.today().replace(day=1)]
         else:
             pl_base_hist = pl_base_hist[pl_base_hist['Mês'] < data_corte_mes]
    
    pl_mes_vigente = pd.concat([pl_base_hist, base_hoje], axis=0)
    
    # Ajustes finais de nomes no histórico consolidado
    replacements = {
        "Murilo Luiz Silva Gino": "Fernando Domingues da Silva",
        "RODRIGO DE MELLO DELIA": "RODRIGO DE MELLO D’ELIA",
        "ROSANA PAVANI": "ROSANA APARECIDA PAVANI DA SILVA",
        "FERNANDO DOMINGUES": "FERNANDO DOMINGUES DA SILVA"
    }
    pl_mes_vigente['Assessor'] = pl_mes_vigente['Assessor'].replace(replacements)
    pl_mes_vigente['Assessor'] = pl_mes_vigente['Assessor'].astype(str).str.upper()
    pl_mes_vigente['PL'] = pl_mes_vigente['PL'].fillna(0)
    pl_mes_vigente['CONTA'] = pl_mes_vigente['CONTA'].astype(str)
    
    # Converte coluna Mês para datetime para padronizar no SQL
    pl_mes_vigente['Mês'] = pd.to_datetime(pl_mes_vigente['Mês'])
    
    pl_mes_vigente.drop_duplicates(subset=["CONTA", "Mês"], keep='first', inplace=True)
    
    AzureLoader.enviar_df(
        df=pl_mes_vigente,
        nome_tabela="PL Base",
        if_exists="replace",
        schema=SCHEMA_DEFAULT
    )

except Exception as e:
    print(f"Erro em PL Base: {e}")

[AzureLoader] Lendo: dbo.PL Base...
[AzureLoader] Lendo: dbo.offshore_adicionar_pl_mes_vigente...
[AzureLoader] Subindo tabela: dbo.offshore_adicionar_pl_mes_vigente (535 linhas)...
[AzureLoader] Chunksize calculado: 522 linhas/lote (Colunas: 4).
[AzureLoader] Concluido: offshore_adicionar_pl_mes_vigente atualizada.
[AzureLoader] Subindo tabela: dbo.PL Base (76930 linhas)...
[AzureLoader] Chunksize calculado: 522 linhas/lote (Colunas: 4).
[AzureLoader] Concluido: PL Base atualizada.


In [14]:
print("\n--- Atualizando Horário de Execução ---")
horario_atualizacao = pd.DataFrame([datetime.now().strftime("%d/%m/%Y %H:%M:%S")], columns=["Horario"])
AzureLoader.enviar_df(df=horario_atualizacao, nome_tabela="horario_atualizacao_relatorios", if_exists="replace", schema=SCHEMA_DEFAULT)


--- Atualizando Horário de Execução ---
[AzureLoader] Subindo tabela: dbo.horario_atualizacao_relatorios (1 linhas)...
[AzureLoader] Chunksize calculado: 1000 linhas/lote (Colunas: 1).
[AzureLoader] Concluido: horario_atualizacao_relatorios atualizada.


In [15]:
tempo_fim = datetime.now().strftime('%H:%M:%S')
ctypes.windll.user32.MessageBoxW(0, f"O pipeline de dados foi concluído com sucesso às {tempo_fim}.", "ETL Finalizado", 0x40)

1